In [84]:
filepath = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/"
filepath

'/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/'

In [85]:
import pretty_midi
import os

def extract_guitar_and_drums(midi_file):
    mid = pretty_midi.PrettyMIDI(midi_file)
    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []
    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))
        if (instrument.program >= 25) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))
    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]
    song_title = os.path.splitext(os.path.basename(file))[0]
    song_dict = {"title": song_title,
                 "guitar": guitar_track,
                 "drums": drum_track
                }
    return song_dict

In [131]:
import os 

directory = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/"


midi_files = []

for filename in os.listdir(directory):
    full_path = os.path.join(directory, filename)
    midi_files.append(full_path)

In [214]:
print(midi_files)

['/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_The_Things_We_Do_for_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Im_Not_In_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/ABC_-_The_Look_of_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/1910_Fruitgum_Company_-_Simon_Says.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/Aaron_Neville_-_Tell_It_Like_It_Is.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/911_-_The_Day_We_Find_Love.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Dreadlock_Holiday.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/38_Special_-_Fantasy_Girl.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/ABC_-_Poison_Arrow.mid', '/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/38_Special_-_Caught_Up_In_You.mid']


In [215]:

list_ = []

for file in midi_files:
    dict_ = extract_guitar_and_drums(full_path)
    list_.append(dict_)


In [218]:
for element in list_:
    print(element)

{'title': '10cc_-_The_Things_We_Do_for_Love', 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"), 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}
{'title': '10cc_-_Im_Not_In_Love', 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"), 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}
{'title': 'ABC_-_The_Look_of_Love', 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"), 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}
{'title': '1910_Fruitgum_Company_-_Simon_Says', 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"), 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}
{'title': 'Aaron_Neville_-_Tell_It_Like_It_Is', 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"), 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}
{'title': '911_-_The_Day_We_Find_Love', 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"), 'drums': Instrument(progra

In [133]:
len(list_)

10

In [134]:
list_[-1]

{'title': '38_Special_-_Caught_Up_In_You',
 'guitar': Instrument(program=25, is_drum=False, name="SteelGtr"),
 'drums': Instrument(program=16, is_drum=True, name="Rhythm&SE")}

In [185]:
list_of_string = []

for element in list_:
    string = ""
    for note in element["guitar"].notes:
        delta = "delta=" + str(note.end - note.start) + " "
        #start_time = "start_time=" + str(int(note.start)) + " "
        #end_time = "end_time=" + str(int(note.end)) + " "
        pitch = "pitch=" + str(note.pitch) + " "
        velocity = "velocity=" + str(note.velocity) + " "
        string = string + delta + pitch + velocity
    list_of_string.append(string)


In [186]:
len(list_of_string)

10

In [187]:
note = dict_["guitar"].notes[0].start
type(note)

numpy.float64

In [188]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer

from transformers import PreTrainedTokenizerFast


tokenizer1 = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer1.pre_tokenizer = WhitespaceSplit()
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

string = list_of_string[0]
tokenizer1.train_from_iterator(string, trainer=trainer)
tokenizer1.save("tokenizer.json")

tokenizer1 = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")
tokenizer1.add_special_tokens({'pad_token': '[PAD]'})

0

In [189]:
len(tokenizer1.get_vocab())

29

In [190]:
tokenizer1.get_vocab()

{'4': 24,
 'e': 9,
 '[SEP]': 2,
 '2': 26,
 'a': 17,
 '[MASK]': 4,
 '9': 14,
 'h': 19,
 '.': 16,
 '[UNK]': 0,
 '3': 27,
 '1': 13,
 'o': 20,
 'p': 21,
 '=': 6,
 't': 7,
 'i': 10,
 '6': 5,
 'l': 11,
 'v': 22,
 'y': 23,
 '8': 28,
 '5': 25,
 '[PAD]': 3,
 '0': 15,
 'd': 18,
 'c': 8,
 '[CLS]': 1,
 '7': 12}

In [191]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer

from transformers import PreTrainedTokenizerFast


tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = WhitespaceSplit()
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

def get_training_corpus():
    for element in list_of_string:
        yield element
    
training_corpus = get_training_corpus()
tokenizer.train_from_iterator(training_corpus, trainer=trainer)
tokenizer.save("tokenizer.json")

tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")

In [192]:
len(tokenizer.get_vocab())

33

In [193]:
tokenizer.get_vocab()

{'[PAD]': 3,
 'delta=0.5': 22,
 'delta=0.3333333333333428': 32,
 'pitch=69': 28,
 'delta=0.5729166666666572': 30,
 'pitch=71': 20,
 'delta=0.1666666666666714': 6,
 'delta=0.16666666666666607': 19,
 'pitch=42': 27,
 'pitch=66': 14,
 'delta=0.16666666666666785': 13,
 'pitch=52': 23,
 '[SEP]': 2,
 'delta=0.65625': 26,
 'pitch=61': 11,
 'pitch=54': 16,
 '[CLS]': 1,
 'delta=0.1666666666666643': 8,
 'pitch=56': 15,
 'pitch=63': 24,
 'delta=0.5833333333333428': 29,
 'pitch=51': 31,
 '[UNK]': 0,
 'pitch=57': 12,
 'pitch=64': 9,
 'pitch=47': 18,
 'pitch=58': 21,
 'velocity=79': 5,
 'pitch=49': 25,
 'delta=0.1666666666666572': 7,
 'pitch=68': 17,
 'pitch=59': 10,
 '[MASK]': 4}

In [194]:
string

'delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=64 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=64 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=64 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=71 velocity=79 delta=0.16666666666666607 pitch=63 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.16666666666666607 pitch=61 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.16666666666666607 pitch=61 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.16666666666666607 pitch=61 velocity=79 delta=0.16666666666666607 pitch=68 velocity=79 delta=0.1666

## Check the range of values that we have

In [180]:
delta = []
pitch = []
velocity = []

for element in list_:
    for note in element["guitar"].notes:
        delta.append(note.end - note.start)
        pitch.append(note.pitch)
        velocity.append(note.velocity)


delta = set(delta)
pitch = set(pitch)
velocity = set(velocity)

In [181]:
delta

{0.1666666666666572,
 0.1666666666666643,
 0.16666666666666607,
 0.16666666666666785,
 0.1666666666666714,
 0.3333333333333428,
 0.5,
 0.5729166666666572,
 0.5833333333333428,
 0.65625}

In [182]:
pitch

{42, 47, 49, 51, 52, 54, 56, 57, 58, 59, 61, 63, 64, 66, 68, 69, 71}

In [183]:
velocity

{79}

In [202]:
delta = []
pitch = []
velocity = []
name = ""
for note in list_[1]["guitar"].notes:
    delta.append(note.end - note.start)
    pitch.append(note.pitch)
    velocity.append(note.velocity)

name = list_[0]["title"]
delta = set(delta)
pitch = set(pitch)
velocity = set(velocity)

In [203]:
name

'10cc_-_The_Things_We_Do_for_Love'

In [204]:
delta

{0.1666666666666572,
 0.1666666666666643,
 0.16666666666666607,
 0.16666666666666785,
 0.1666666666666714,
 0.3333333333333428,
 0.5,
 0.5729166666666572,
 0.5833333333333428,
 0.65625}

In [205]:
pitch

{42, 47, 49, 51, 52, 54, 56, 57, 58, 59, 61, 63, 64, 66, 68, 69, 71}

In [206]:
velocity

{79}

In [212]:
list_[7]["guitar"].notes[-1]

Note(start=187.750000, end=187.916667, pitch=47, velocity=79)

In [213]:
list_[5]["guitar"].notes[-1]

Note(start=187.750000, end=187.916667, pitch=47, velocity=79)